In [2]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
full_dat = pd.read_csv("data/merged.csv")
full_dat['r_min dis'] = full_dat['r_min dis'].str.replace('[','').str.replace(']','').astype('float')
full_dat['capacity'] = np.where(full_dat['r_min dis']<1, full_dat['capacity'] ,'0')
full_dat = full_dat[["road num","length_km","volume","demandfactor","capacity"]]
full_dat = full_dat.dropna()
full_dat['capacity'] = full_dat['capacity'].str.replace('kW','').astype('int')
full_dat['demand'] = full_dat['length_km']*full_dat['demandfactor']
full_dat = full_dat[["road num","length_km","demand","capacity"]]
full_dat.columns = ['road','length','demand','capacity']
full_dat['ev'] = full_dat.groupby(['road']).cumcount()+1
full_dat['ev'] = 'r'+full_dat['road'].astype('str')+'_e'+full_dat['ev'].astype('str')
full_dat['road'] = full_dat['road'].astype('str')
full_dat.head()

/home/expero/env/or6333/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (17,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/expero/env/or6333/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,road,length,demand,capacity,ev
143,2006,14,4.975008,0,r2006_e1
144,2006,14,4.975008,0,r2006_e2
145,2006,14,4.975008,0,r2006_e3
146,2006,14,4.975008,0,r2006_e4
147,2006,14,4.975008,0,r2006_e5


In [3]:
# Given
min_length = 25
new_ev_cap = 150
# d = pd.DataFrame({'road':['r1','r1','r2','r2','r3'], 'length':[10,10,20,20,30], 'ev':['e1','e2','e3','e4','e5'], 'demand':[100,100,200,200,100], 'capacity':[40,50,40,30,30]})
# d = pd.DataFrame({'road':['r1','r1','r2','r2'], 'length':[10,10,20,20], 'ev':['e1','e2','e3','e4'], 'demand':[100,100,200,200], 'capacity':[40,50,60,60]})
# d = pd.DataFrame({'road':['r1','r1'], 'length':[10,10], 'ev':['e1','e2'], 'demand':[100,100], 'capacity':[40,50]})
d = full_dat
d['max_road_capacity'] = (d['length']//min_length)*new_ev_cap
d['demand'] = np.where(d['demand']<d['max_road_capacity'], d['demand'], d['max_road_capacity'])
d

,road,length,demand,capacity,ev,max_road_capacity
143,2006,14,0.000000,0,r2006_e1,0
144,2006,14,0.000000,0,r2006_e2,0
145,2006,14,0.000000,0,r2006_e3,0
146,2006,14,0.000000,0,r2006_e4,0
147,2006,14,0.000000,0,r2006_e5,0
...,...,...,...,...,...,...
130729,53799,8,0.000000,0,r53799_e5,0
130730,53800,16,0.000000,0,r53800_e7,0
130731,53801,83,198.503959,0,r53801_e24,450
130732,53802,20,0.000000,0,r53802_e6,0


### Decision Variables

In [4]:
new_station = {}
d2 = d[['road', 'length']].drop_duplicates()
for i in range(d2.shape[0]):
    for j in range(d2['length'].iloc[i] // min_length):
        var_name = 'new_ev_'+d2['road'].iloc[i]+'_'+str(j)

        if d2['road'].iloc[i] in new_station.keys():
            new_station[d2['road'].iloc[i]].append(LpVariable(var_name, lowBound=0, cat="Integer"))
        else:
            new_station[d2['road'].iloc[i]] = [LpVariable(var_name, lowBound=0, cat="Integer")]
new_station

{'1995': [new_ev_1995_0, new_ev_1995_1, new_ev_1995_2],
 '2726': [new_ev_2726_0,
  new_ev_2726_1,
  new_ev_2726_2,
  new_ev_2726_3,
  new_ev_2726_4,
  new_ev_2726_5,
  new_ev_2726_6],
 '2725': [new_ev_2725_0],
 '49422': [new_ev_49422_0, new_ev_49422_1],
 '4144': [new_ev_4144_0],
 '2719': [new_ev_2719_0,
  new_ev_2719_1,
  new_ev_2719_2,
  new_ev_2719_3,
  new_ev_2719_4,
  new_ev_2719_5,
  new_ev_2719_6],
 '49378': [new_ev_49378_0, new_ev_49378_1, new_ev_49378_2],
 '2714': [new_ev_2714_0],
 '3571': [new_ev_3571_0],
 '51648': [new_ev_51648_0, new_ev_51648_1, new_ev_51648_2],
 '49307': [new_ev_49307_0, new_ev_49307_1, new_ev_49307_2],
 '49404': [new_ev_49404_0],
 '51677': [new_ev_51677_0],
 '2698': [new_ev_2698_0, new_ev_2698_1],
 '2700': [new_ev_2700_0, new_ev_2700_1],
 '3635': [new_ev_3635_0],
 '2708': [new_ev_2708_0, new_ev_2708_1],
 '2697': [new_ev_2697_0, new_ev_2697_1],
 '2709': [new_ev_2709_0, new_ev_2709_1, new_ev_2709_2],
 '4145': [new_ev_4145_0, new_ev_4145_1],
 '2739': [new_ev_

### Objective Function

In [5]:
prob = LpProblem("EV_Station_Optimisation", LpMinimize)
terms = []
for k in new_station.keys():
    for n in new_station[k]:
        terms += [n]
prob += lpSum(terms)
prob

EV_Station_Optimisation:
MINIMIZE
1*new_ev_1002_0 + 1*new_ev_1002_1 + 1*new_ev_1002_2 + 1*new_ev_1002_3 + 1*new_ev_1002_4 + 1*new_ev_1003_0 + 1*new_ev_1003_1 + 1*new_ev_1005_0 + 1*new_ev_1005_1 + 1*new_ev_1005_2 + 1*new_ev_1005_3 + 1*new_ev_1005_4 + 1*new_ev_1005_5 + 1*new_ev_1006_0 + 1*new_ev_1006_1 + 1*new_ev_1006_2 + 1*new_ev_1006_3 + 1*new_ev_1006_4 + 1*new_ev_1006_5 + 1*new_ev_1006_6 + 1*new_ev_1007_0 + 1*new_ev_1007_1 + 1*new_ev_1007_2 + 1*new_ev_1007_3 + 1*new_ev_1007_4 + 1*new_ev_1007_5 + 1*new_ev_1008_0 + 1*new_ev_1008_1 + 1*new_ev_1008_2 + 1*new_ev_1008_3 + 1*new_ev_1008_4 + 1*new_ev_1008_5 + 1*new_ev_1008_6 + 1*new_ev_1008_7 + 1*new_ev_1008_8 + 1*new_ev_1008_9 + 1*new_ev_1009_0 + 1*new_ev_1009_1 + 1*new_ev_1009_2 + 1*new_ev_1009_3 + 1*new_ev_1009_4 + 1*new_ev_1009_5 + 1*new_ev_1009_6 + 1*new_ev_1009_7 + 1*new_ev_100_0 + 1*new_ev_100_1 + 1*new_ev_100_2 + 1*new_ev_1010_0 + 1*new_ev_1010_1 + 1*new_ev_1010_2 + 1*new_ev_1010_3 + 1*new_ev_1010_4 + 1*new_ev_1011_0 + 1*new_ev_1011_1

### Constraints

In [6]:
# constraint to have more than 0 new EVs
for i,k in enumerate(new_station.keys()):
    prob += lpSum(ev for ev in new_station[k]) >= 0

    for ev in new_station[k]:
        prob += ev<=new_ev_cap

for i,k in enumerate(new_station.keys()):
    term1 = lpSum(ev for ev in new_station[k])
    exiting_r_ev = d[d['road']==k]
    term2 = lpSum(ec for ec in exiting_r_ev['capacity'])
    prob += term1 + term2 >= (exiting_r_ev['demand'].unique()[0])

prob

EV_Station_Optimisation:
MINIMIZE
1*new_ev_1002_0 + 1*new_ev_1002_1 + 1*new_ev_1002_2 + 1*new_ev_1002_3 + 1*new_ev_1002_4 + 1*new_ev_1003_0 + 1*new_ev_1003_1 + 1*new_ev_1005_0 + 1*new_ev_1005_1 + 1*new_ev_1005_2 + 1*new_ev_1005_3 + 1*new_ev_1005_4 + 1*new_ev_1005_5 + 1*new_ev_1006_0 + 1*new_ev_1006_1 + 1*new_ev_1006_2 + 1*new_ev_1006_3 + 1*new_ev_1006_4 + 1*new_ev_1006_5 + 1*new_ev_1006_6 + 1*new_ev_1007_0 + 1*new_ev_1007_1 + 1*new_ev_1007_2 + 1*new_ev_1007_3 + 1*new_ev_1007_4 + 1*new_ev_1007_5 + 1*new_ev_1008_0 + 1*new_ev_1008_1 + 1*new_ev_1008_2 + 1*new_ev_1008_3 + 1*new_ev_1008_4 + 1*new_ev_1008_5 + 1*new_ev_1008_6 + 1*new_ev_1008_7 + 1*new_ev_1008_8 + 1*new_ev_1008_9 + 1*new_ev_1009_0 + 1*new_ev_1009_1 + 1*new_ev_1009_2 + 1*new_ev_1009_3 + 1*new_ev_1009_4 + 1*new_ev_1009_5 + 1*new_ev_1009_6 + 1*new_ev_1009_7 + 1*new_ev_100_0 + 1*new_ev_100_1 + 1*new_ev_100_2 + 1*new_ev_1010_0 + 1*new_ev_1010_1 + 1*new_ev_1010_2 + 1*new_ev_1010_3 + 1*new_ev_1010_4 + 1*new_ev_1011_0 + 1*new_ev_1011_1

In [7]:
prob.solve()
LpStatus[prob.status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/expero/env/or6333/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/85131e52eac440709bc8364698091f03-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/85131e52eac440709bc8364698091f03-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16227 COLUMNS
At line 72052 RHS
At line 88275 BOUNDS
At line 97580 ENDATA
Problem MODEL has 16222 rows, 9304 columns and 27912 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 514091 - 0.00 seconds
Cgl0004I processed model has 208 rows, 649 columns (649 integer (0 of which binary)) and 649 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 515341 found by greedy cover after 0 iterations and 0 nodes (0.08 seconds)
Cbc0038I Full problem 208 rows 649 columns, reduced to 49 rows 98 column

'Optimal'

In [8]:
print('Optimization Solution:')
for var in prob.variables():
    print("Value of binary variable, " + str(var) +" : "+ str(var.varValue))
            
# print the costs of the optimal diet             
print("Total Optimal objective value = " + str(value(prob.objective)))

Optimization Solution:
Value of binary variable, new_ev_1002_0 : 150.0
Value of binary variable, new_ev_1002_1 : 150.0
Value of binary variable, new_ev_1002_2 : 150.0
Value of binary variable, new_ev_1002_3 : 150.0
Value of binary variable, new_ev_1002_4 : 150.0
Value of binary variable, new_ev_1003_0 : 150.0
Value of binary variable, new_ev_1003_1 : 150.0
Value of binary variable, new_ev_1005_0 : 94.0
Value of binary variable, new_ev_1005_1 : 150.0
Value of binary variable, new_ev_1005_2 : 150.0
Value of binary variable, new_ev_1005_3 : 150.0
Value of binary variable, new_ev_1005_4 : 150.0
Value of binary variable, new_ev_1005_5 : 149.0
Value of binary variable, new_ev_1006_0 : 150.0
Value of binary variable, new_ev_1006_1 : 150.0
Value of binary variable, new_ev_1006_2 : 150.0
Value of binary variable, new_ev_1006_3 : 150.0
Value of binary variable, new_ev_1006_4 : 150.0
Value of binary variable, new_ev_1006_5 : 150.0
Value of binary variable, new_ev_1006_6 : 35.0
Value of binary var

In [9]:
cols = ['new_ev','capacity']
res = pd.DataFrame(columns = cols, index=range(len(prob.variables())))
for i,var in enumerate(prob.variables()):
    res.loc[i].new_ev = str(var)
    res.loc[i].capacity = var.varValue
res

,new_ev,capacity
0,new_ev_1002_0,150.0
1,new_ev_1002_1,150.0
2,new_ev_1002_2,150.0
3,new_ev_1002_3,150.0
4,new_ev_1002_4,150.0
...,...,...
9299,new_ev_999_1,0.0
9300,new_ev_999_2,0.0
9301,new_ev_999_3,0.0
9302,new_ev_999_4,0.0


In [10]:
res.to_csv('new_ev.csv',index=False)

In [23]:
temp = pd.read_csv("data/merged.csv")
temp.columns

/home/expero/env/or6333/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (17,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['index', 'lat', 'lon', 'road num', 'type', 'name', 'length_km', 'toll',
       'arg_min', 'volume', 'demandfactor', 'state', 'Fuel Type Code',
       'Station Name', 'Street Address', 'City', 'State', 'ZIP',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Open Date', 'EV Connector Types', 'Country', 'Access Code',
       'Access Detail Code', 'Facility Type', 'EV Pricing',
       'EV On-Site Renewable Source', 'Restricted Access', 'capacity', 'ports',
       'r_index', 'r_lat', 'r_lon', 'r_road num', 'r_type', 'r_name',
       'r_length_km', 'r_toll', 'r_arg_min', 'r_volume', 'r_demandfactor',
       'r_state', 'r_min dis'],
      dtype='object')

In [24]:
temp['capacity'] = temp['capacity'].str.replace('kW','').astype('float')
temp = temp[['State','City','capacity']]
temp['State'] = temp['State'].str.upper()
temp = temp.dropna()
temp.head()

,State,City,capacity
8,AZ,Glendale,100.0
9,VT,Newport,15.0
66,PR,San Juan,15.0
67,PR,San Juan,15.0
77,CA,Carolina,15.0


In [25]:
temp['State'].unique()

array(['AZ', 'VT', 'PR', 'CA', 'FL', 'HI', 'WA', 'TX', 'LA', 'MS', 'AL',
       'GA', 'NM', 'SC', 'AR', 'MD', 'OK', 'NC', 'QC', 'TN', 'NV', 'OR',
       'MO', 'VA', 'KY', 'KS', 'UT', 'CO', 'WV', 'IL', 'IN', 'DE', 'OH',
       'DC', 'NJ', 'NY', 'PA', 'NE', 'IA', 'CT', 'WY', 'MA', 'RI', 'MI',
       'WI', 'ID', 'NH', 'SD', 'ME', 'ON', 'MN', 'MT', 'ND', 'AK'],
      dtype=object)

In [26]:
temp_g = temp.groupby(['State','City']).agg(total_capacity=('capacity','sum')).reset_index()
temp_g = temp_g.sort_values(['State','total_capacity'],ascending=False)
temp_g['rank'] = temp_g.groupby(['State']).cumcount()+1
temp_g['rank'] = np.where(temp_g['rank'] < 5, temp_g['rank'], 0)
temp_g['rank'] = temp_g['rank'].astype('str')
temp_g['rank'] = np.where(temp_g['rank'] != '0', temp_g['rank'], 'Others')
temp_g['City'] = np.where(temp_g['rank'] != 'Others', temp_g['City'], 'Others')
# print(temp_g['rank'].unique())
temp_g

,State,City,total_capacity,rank
6967,WY,Jackson,1113.0,1
6977,WY,Rawlins,1000.0,2
6962,WY,Evanston,600.0,3
6970,WY,Laramie,460.0,4
6979,WY,Others,437.0,Others
...,...,...,...,...
4,AK,Others,15.0,Others
7,AK,Others,15.0,Others
5,AK,Others,7.0,Others
11,AK,Others,7.0,Others


In [27]:
plot1 = temp_g.groupby(['State','City','rank']).agg(total_capacity=('total_capacity','sum')).reset_index()
plot1.to_csv('state_top4_capacity.csv',index=False)
plot1

,State,City,rank,total_capacity
0,AK,Anchorage,3,132.0
1,AK,Homer,4,114.0
2,AK,Juneau,1,420.0
3,AK,Others,Others,666.0
4,AK,Soldotna,2,329.0
...,...,...,...,...
256,WY,Evanston,3,600.0
257,WY,Jackson,1,1113.0
258,WY,Laramie,4,460.0
259,WY,Others,Others,2443.0


In [11]:
res['road num'] = res['new_ev'].str.split('_', expand=True).iloc[:,2]
res_g = res.groupby(['road num']).agg(total_proposed_cap = ('capacity','sum')).reset_index()
res_g

NameError: name 'res' is not defined

In [57]:
temp2 = pd.read_csv("data/merged.csv")
temp2 = temp2[['road num','State', 'City']].drop_duplicates().dropna()
temp2

/home/expero/env/or6333/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (17,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,road num,State,City
8,2833,AZ,Glendale
9,2833,VT,Newport
66,2801,PR,San Juan
67,2803,PR,San Juan
77,2808,CA,Carolina
...,...,...,...
122906,1064,AK,Cantwell
122912,1068,AK,Denali National Park and Preserve
122960,1068,AK,Denali Park
122976,1063,AK,Healy
